Multiple Replications Procedure for single-dimensional newsvendor problem

In [1]:
import numpy as np
import pandas as pd
from scipy import stats as st
import cvxpy as cp

In [2]:
df2 = pd.read_excel (r'demand.xlsx')

In [3]:
data = df2.iloc[1:,2:]
#data

In [4]:
# cost parameters
c = 0.5 # purchase cost
s = 1 # sales cost

In [5]:
# MRP parameters
alpha = 0.05 # confidence
xhat = 0.75 # order quantity

In [6]:
def PN_solve(demand, s, c, j):
    x = cp.Variable(1)
    term = 0
    for i in np.arange(0,demand.shape[0], 1):
        term += (s * ((x + demand.iloc[i,j] - cp.abs(x - demand.iloc[i,j]))/2) - c * x)
    #print (term)
    objective = cp.Maximize((1/demand.shape[0]) * term)
    prob = cp.Problem(objective)
    result = prob.solve()
    #print("The optimum is reached at: ", result)
    #print("The optimal values for x are: ", x.value)
    return(x.value)

In [7]:
def Gjnx_calc(demand, xstar, xhat, j):
    term = 0
    for i in np.arange(0,demand.shape[0], 1):
        term += ((s * ((xhat + demand.iloc[i,j] - abs(xhat - demand.iloc[i,j]))/2) - c * xhat) - 
                 (s * ((xstar + demand.iloc[i,j] - abs(xstar - demand.iloc[i,j]))/2) - c * xstar))
    #print(term)
    return(term)

In [8]:
# define MRP function for single-dimensional newsvendor problem
def MRP_NV(alpha, xhat, data):
    
    # step 1
    buy = np.empty([1, data.shape[1]])
    Gjnx = np.empty([1, data.shape[1]])
    for j in np.arange(0,data.shape[1],1):
        # step 1.a -> solve P(n)
        xstar = PN_solve(data, s, c, j)
        buy[0,j] = xstar
        # step 1.b -> calculate Gnj(xhat)
        Gjnx[0,j] = (1/data.shape[0]) * Gjnx_calc(data, xstar, xhat, j) 
        
    # step 2
    # calculate sample mean
    Gbar_nm_xhat = np.mean(Gjnx)
    # calculate sample variance
    Var_nm_xhat = np.var(Gjnx, ddof = 1)
    
    # step 3
    CI = [Gbar_nm_xhat + st.t.ppf(1-alpha, data.shape[1]-1) * ((np.sqrt(Var_nm_xhat))/(np.sqrt(data.shape[1]))), 0]
    print("CI:", CI)
    return(CI)

In [9]:
y = MRP_NV(alpha, xhat, data)

CI: [-0.0014218131006010493, 0]
